## Trajectory Optimization with Differentiable Physics

### Imports

In [ ]:
from copy import deepcopy
# math
import numpy as np
# physics
import warp as wp
import warp.sim as wps
# graphics
import mitsuba as mi
from mitsuba.scalar_rgb import Transform4f as mit
# plots
from matplotlib import pyplot as plt

# setup
wp.init()
mi.set_variant('llvm_ad_rgb')
print(f'Mitsuba 3 Variant: {mi.variant()}')

### Setup

In [ ]:
#########
# Scene #
#########

# camera positioned to view the scene, adjust as needed
cam_pos = np.array([0.0, 1.0, 5.0])  
cam_target = np.array([0.0, 1.0, 0.0])
cam_up = np.array([0.0, 1.0, 0.0])

# particle
particle_init_pos = np.array([-0.5, 1.0, 0.0])
particle_init_vel = np.array([5.0, -5.0, 0.0])
particle_radius = 0.2
particle_mass = 1.0
particle_rgb = np.array([0.2, 0.25, 0.7])

# wall
wall_pos = np.array([2.0, 1.0, 0.0])
wall_extents = np.array([0.25, 1.0, 1.0])
wall_rgb = np.array([0.7, 0.25, 0.2])

# target
target_pos = np.array([-2.0, 1.5, 0.0])

# contact parameters
ke = 1.0e4
kf = 0.0
kd = 1.0e1
mu = 0.2
rt = 1.0
margin = 10.0

# simulation parameters
sim_duration = 0.6
# control frequency
fps = 60
frame_dt = 1.0 / float(fps)
frame_steps = int(sim_duration / frame_dt)
# sim frequency
sim_substeps = 8
sim_steps = frame_steps * sim_substeps
sim_dt = frame_dt / sim_substeps

In [ ]:
##############
# Simulation #
##############

@wp.func
def square_loss(pos: wp.vec3, target: wp.vec3):
    delta = target - pos
    #return wp.sqrt(wp.dot(delta, delta))
    return wp.dot(delta, delta)

@wp.kernel
def eval(pred: wp.array(dtype=wp.vec3), target: wp.vec3, loss: wp.array(dtype=float)):
    i = wp.tid()
    loss[i] = square_loss(pred[i], target)

@wp.kernel
def step(x: wp.array(dtype=wp.vec3), grad: wp.array(dtype=wp.vec3), alpha: float):
    tid = wp.tid()

    # gradient descent step
    x[tid] = x[tid] - grad[tid] * alpha


def build_phys(device):
    builder = wps.ModelBuilder()

    # particle
    builder.add_particle(pos=particle_init_pos, vel=particle_init_vel, radius=particle_radius, mass=particle_mass)
    # wall
    builder.add_shape_box(body=-1, 
                          pos=wall_pos, 
                          hx=wall_extents[0], hy=wall_extents[1], hz=wall_extents[2], 
                          ke=ke, kf=kf, kd=kd, mu=mu)
    
    model = builder.finalize(device, requires_grad=True)

    # ground
    model.soft_contact_kf = kf
    model.soft_contact_margin = margin

    # allocate sim states for trajectory
    states = [model.state(requires_grad=True) for _ in range(sim_steps + 1)]

    # one-shot contact creation
    wp.sim.collide(model, states[0])

    return model, states


def run_phys(device, model, integrator, states):
    wp.capture_begin(device=device)
    tape = wp.Tape()
    with tape:
        for i in range(sim_steps):
            states[i].clear_forces()
            integrator.simulate(model, states[i], states[i + 1], sim_dt)
    graph = wp.capture_end()

    return graph, tape


def run_loss(device, loss, pred, target):
    wp.capture_begin(device=device)
    tape = wp.Tape()
    with tape:
        wp.launch(eval, dim=pred.shape[0], inputs=[pred, target, loss], device=device)
    graph = wp.capture_end()

    return graph, tape
    

def get_trajectory(states):
    trajectory = []
    for state in states[::sim_substeps]:
        trajectory.append(state.particle_q.numpy().squeeze())
    return np.array(trajectory)


def jacobian_phys(tape, params, pred, pred_dim=3, params_dim=3):
    Jp = np.zeros((pred.shape[0], pred_dim, pred.shape[0], params_dim), dtype=np.float32)
    for obj_index in range(pred.shape[0]):
        for dim_index in range(pred_dim):
            select_index = np.zeros((pred.shape[0], pred_dim))
            select_index[obj_index, dim_index] = 1.0
            e = wp.array(select_index, dtype=wp.vec3)
            # pass input gradients to the output buffer to apply selection
            tape.backward(grads={pred: e})
            grad_val = tape.gradients[params]
            Jp[obj_index, dim_index, ...] = grad_val.numpy()
            tape.zero()
    # I shouldnt need to do this, but for some reason the gradients are not zeroed
    pred.grad.zero_()
    tape.gradients[pred].zero_()
    
    return Jp


def jacobian_loss(tape, loss, pred, pred_dim=3):
    Jl = np.zeros((pred.shape[0], pred_dim), dtype=np.float32)
    for i in range(pred.shape[0]):
        # select which row of the Jacobian we want to compute
        select_index = np.zeros((pred.shape[0],))
        select_index[i] = 1.0
        e = wp.array(select_index, dtype=wp.float32)
        # pass input gradients to the output buffer to apply selection
        tape.backward(grads={loss: e})
        Jl[i] = tape.gradients[pred].numpy()
        tape.zero()
    loss.grad.zero_() 
    tape.gradients[loss].zero_()
    
    return Jl


def grad(Jp, Jl):
    grad = np.zeros((Jp.shape[2], Jp.shape[3]), dtype=np.float32)

    for obj_index in range(Jp.shape[2]):
        for dim_index in range(Jp.shape[3]):
            grad[obj_index, dim_index] = np.dot(Jl.squeeze(), Jp[obj_index, dim_index].squeeze())
    
    return grad

### States

In [ ]:
#############
# Rendering #
#############

img_width = 128
img_height = 128
fov = 60

render_integrator = {
    'type': 'direct_reparam',
}

scene_dict = {
    'type': 'scene',
    'integrator': render_integrator,
    'sensor':  {
        'type': 'perspective',
        'to_world': mit.look_at(
                        origin=cam_pos,
                        target=cam_target,
                        up=cam_up),
        'fov': fov,
        'film': {
            'type': 'hdrfilm',
            'width': img_width,
            'height': img_height,
            'rfilter': { 'type': 'gaussian' },
            'sample_border': True
        },
    },
    "light": {
        "type": "constant"
    },
    'particle': {
        'type': 'sphere',
        'center': particle_init_pos,
        'radius': particle_radius,
        'bsdf': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': particle_rgb
            }
        }
    },
    'wall': {
        'type': 'cube',
        'to_world': mit.translate(wall_pos).scale(wall_extents),
        'bsdf' : {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': wall_rgb
            }
        }
    },
    "ground": {
        "type": "rectangle",
        "to_world": mit.rotate(axis=[1.0, 0.0, 0.0], angle=-90).scale(20),
        "bsdf": {
            "type": "diffuse",
            'reflectance': {
                'type': 'checkerboard',
                'to_uv': mi.ScalarTransform4f.scale([10, 10, 1])
            }
        }
    }
}


fig, axs = plt.subplots(1, 2, figsize=(10, 5))

##########
# States #
##########

init_scene_dict = dict(scene_dict)
init_scene_dict['particle']['center'] = deepcopy(particle_init_pos)
bitmap_init = mi.util.convert_to_bitmap( mi.render(mi.load_dict(init_scene_dict), spp=16))
axs[0].imshow(bitmap_init)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].set_title('Initial State')

target_scene_dict = dict(scene_dict)
target_scene_dict['particle']['center'] = deepcopy(target_pos)
bitmap_target = mi.util.convert_to_bitmap(mi.render(mi.load_dict(target_scene_dict), spp=16))
axs[1].imshow(bitmap_target)
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].set_title('Target State')

plt.show()
#plt.savefig('fig_direct_traj_states.pdf', bbox_inches='tight')

### Optimization

In [ ]:
train_iters = 250
train_rate = 0.02
train_save_interval = 50

phys_device = wp.get_cuda_devices()[0]
phys_model, phys_states = build_phys(phys_device)
phys_integrator = wp.sim.SemiImplicitIntegrator()

# build physics graph
phys_graph, phys_tape = run_phys(phys_device, phys_model, phys_integrator, phys_states)

# parameters to estimate and representation for loss
x = phys_states[0].particle_qd
phys_pred = phys_states[-1].particle_q

# build loss graph
loss = wp.zeros(phys_pred.shape[0], dtype=wp.float32,  device=phys_device, requires_grad=True)
loss_graph, loss_tape = run_loss(phys_device, loss, phys_pred, target_pos)

# for init metrics
wp.capture_launch(phys_graph)
wp.capture_launch(loss_graph)
# metrics
vel_trajs = [get_trajectory(phys_states)]
vel_losses = [loss.numpy()[0]]

for i in range(train_iters):
    # calculate update
    phys_jacobian = jacobian_phys(phys_tape, x, phys_pred)
    loss_jacobian = jacobian_loss(loss_tape, loss, phys_pred)
    #x_grad = grad(phys_jacobian, loss_jacobian)
    x_grad = wp.array(grad(phys_jacobian, loss_jacobian), dtype=wp.vec3, device=phys_device)
    # update values
    wp.launch(step, dim=phys_model.particle_count, inputs=[x, x_grad, train_rate], device=phys_device)
    # re-run simulation
    wp.capture_launch(phys_graph)
    wp.capture_launch(loss_graph)
    
    vel_losses.append(loss.numpy()[0])
    if (i+1) % train_save_interval == 0:
        print("Iteration: {}, Loss: {}".format(i+1, vel_losses[-1]))
        vel_trajs.append(get_trajectory(phys_states))

### Results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

################
# Trajectories #
################

axs[0].set_title('Trajectories') 
for i, vel_traj in enumerate(vel_trajs):
    axs[0].plot(vel_traj[:, 0], vel_traj[:, 1], label=f'Iter. {i*train_save_interval}')
axs[0].scatter(particle_init_pos[0], particle_init_pos[1], label='start', facecolors='none', edgecolors='black', s=50)
axs[0].scatter(target_pos[0], target_pos[1], color='black', label='target', marker='+', s=50)
axs[0].set_xlim([-2.1, 2.1])
axs[0].set_ylim([0.0, 2.0])
axs[0].set_xlabel('x');
axs[0].set_ylabel('y');
axs[0].legend(loc='lower left' )


##########
# Losses #
##########

axs[1].plot(vel_losses)
axs[1].set_xlabel('Iteration'); 
axs[1].set_ylabel('Loss'); 
axs[1].set_title('Loss per Iteration')

plt.show()
#plt.savefig('fig_direct_traj_optim.pdf', bbox_inches='tight')